In [21]:
%load_ext autoreload
%autoreload 2

from helpers import *

import bz2
import json

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
path = fixpath(QUOTES_2020_LABELED_CLEANED_VARIANTS)

df = pd.read_json(path, orient='records', lines=True)

In [23]:
df

,quoteID,quotation,speaker,date,numOccurrences,id,party_label,US_congress_bio_ID,quotation_cleanA,quotation_cleanB,quotation_cleanC,quotation_cleanD
0,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,sue myrick,2020-01-16 12:00:13,1,Q367796,R,M001134,[ Department of Homeland Security ] was livid ...,department of homeland security was livid and ...,department homeland security livid strongly ur...,depart homeland secur livid strong urg agenda ...
1,2020-01-26-000499,a few of the candidates who will do better in ...,dave loebsack,2020-01-26 13:21:36,11,Q771586,D,L000565,a few of the candidates who will do better in ...,a few of the candidates who will do better in ...,candidates better part world,candid better part world
2,2020-01-26-040663,"The generational thing is important, quite hon...",dave loebsack,2020-01-26 13:21:36,11,Q771586,D,L000565,"The generational thing is important, quite hon...",the generational thing is important quite hone...,generational thing important quite honestly th...,generat thing import quit honest think everyth...
3,2020-01-20-000982,a host of other protections,debbie lesko,2020-01-20 15:32:48,1,Q16731415,R,L000589,a host of other protections,a host of other protections,host protections,host protect
4,2020-01-09-060095,"No, are you kidding? The way that this place l...",debbie lesko,2020-01-09 23:15:21,1,Q16731415,R,L000589,"No, are you kidding? The way that this place l...",no are you kidding the way that this place lea...,kidding way place leaks around,kid way place leak around
...,...,...,...,...,...,...,...,...,...,...,...,...
6650,2020-02-11-099568,VA should take the time it needs to get this $...,mark takano,2020-02-11 13:01:17,2,Q399593,D,T000472,VA should take the time it needs to get this $...,va should take the time it needs to get this b...,va take time needs get billion implementation ...,va take time need get billion implement right ...
6651,2020-04-07-063379,We have heard that COVID-19 is indiscriminate ...,pat robertson,2020-04-07 17:04:04,1,Q364990,R,None,We have heard that COVID- is indiscriminate as...,we have heard that covid is indiscriminate as ...,heard covid indiscriminate gets might prime mi...,heard covid indiscrimin get might prime minist...
6652,2020-02-17-078748,"We have the Wasatch Front, and we have rural U...",scott sandall,2020-02-17 13:00:40,1,Q23772029,R,None,"We have the Wasatch Front, and we have rural U...",we have the wasatch front and we have rural utah,wasatch front rural utah,wasatch front rural utah
6653,2020-01-08-098089,we're not looking for a savior; we're looking ...,dan horrigan,2020-01-08 00:00:00,1,Q22005176,D,None,we're not looking for a savior; we're looking ...,we re not looking for a savior we re looking f...,looking savior looking partner white house,look savior look partner white hous
